# Data Visualization 2 - Madrid Weather Data Visualization

In [1]:
 # import libs
import plotly.express as px
import pandas as pd
import plotly.io as pio
from plotly.subplots import make_subplots
from dash import Dash 
from dash import html, dcc 
from dash import callback, Output, Input
from datetime import datetime
pio.templates.default = 'plotly_white'

In [2]:
# create df for the wine tasting dataset
df = pd.read_csv('https://raw.githubusercontent.com/viethngn/CEU_MSc_BA_ECBS5251_Data_Visualization_2/main/final_project/Madrid_Daily_Weather%201997-2015.csv')
df.head().T;

In [3]:
# filter missing data
df = df[(df['Mean TemperatureC'].notna())]

# add time columns
df['month'] = df.CET.apply(lambda x: int(datetime.strptime(x, '%Y-%m-%d').strftime('%m')))
df['year'] = df.CET.apply(lambda x: int(datetime.strptime(x, '%Y-%m-%d').strftime('%Y')))
df.head().T;

In [4]:
# support function to create dual y-aixs plot
def plotly_dual_axis(data, title="", y1="", y2="", x=""):
    # Create subplot with secondary axis
    subplot_fig = make_subplots(specs=[[{"secondary_y": True}]])

    #Put Dataframe in fig1 and fig2
    fig1 = px.line(data, x=x, y=y1)
    fig2 = px.line(data, x=x, y=y2)
    #Change the axis for fig2
    fig2.update_traces(yaxis="y2")

    #Add the figs to the subplot figure
    subplot_fig.add_traces(fig1.data + fig2.data)

    #FORMAT subplot figure
    subplot_fig.update_layout(title=title, yaxis=dict(title=y1), yaxis2=dict(title=y2), xaxis=dict(title=x))

    #RECOLOR so as not to have overlapping colors
    subplot_fig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))


    return subplot_fig

In [5]:
# create the Dash app
temp_title = "Temperature (Celcius)"
wind_title = "Wind Speed (Km/h)"
time_title = "Date"

fig = plotly_dual_axis(df, title="test", x='CET', y1='Mean TemperatureC', y2=' Mean Wind SpeedKm/h')

app = Dash(__name__)

app.layout = html.Div([
    html.H1('Madrid Weather Dashboard', style={'textAlign':'center', "font-family": "sans-serif", "font-size":"36px"}),
    html.Label('Set the range for year:', style={'font-weight': 'bold', "text-align": "center", 
                                                "font-family": "sans-serif"}),
    dcc.RangeSlider(min=df.year.min(), max=df.year.max(), id='year_range', step=1, value=[2005, 2005],
                    marks={i: '{}'.format(i) for i in range(df.year.min(),df.year.max() + 1)}),
    html.Label('Set the range for month:', style={'font-weight': 'bold', "text-align": "center", 
                                                "font-family": "sans-serif"}),
    dcc.RadioItems( id='plot_mode', options=['Tempurature', 'Wind speed', 'Dual Temp & Wind'], value='Dual Temp & Wind', inline=True, style={"font-family": "sans-serif"}),
    dcc.Graph(id = 'graph', figure = fig)
])

@app.callback(
    Output('graph', 'figure'),
    Input('year_range','value'),
    Input('plot_mode','value')
)

def update(_year_range, _plot_mode):
    if _plot_mode == 'Dual Temp & Wind':
        fig = plotly_dual_axis(df[(df.year >= _year_range[0]) 
                                  & (df.year <= _year_range[1])], 
                               title="test", x='CET', y1='Mean TemperatureC', y2=' Mean Wind SpeedKm/h')
        fig.update_layout(title = "Temperature & Wind Speed by Date", title_x = 0.5)
        fig.update_layout(yaxis=dict(title=temp_title), yaxis2=dict(title=wind_title), xaxis=dict(title=time_title))
    elif _plot_mode == 'Tempurature':
        fig = px.line(df[(df.year >= _year_range[0]) 
                                  & (df.year <= _year_range[1])], 
                      x='CET', y='Mean TemperatureC', 
                      labels={
                          'CET': time_title,
                          'Mean TemperatureC': temp_title
                      })
        fig.update_layout(title = "Temperature by Date", title_x = 0.5)
    else:
        fig = px.line(df[(df.year >= _year_range[0]) 
                                  & (df.year <= _year_range[1])], 
                      x='CET', y=' Mean Wind SpeedKm/h',
                      labels={
                          'CET': time_title,
                          ' Mean Wind SpeedKm/h': wind_title
                      })
        fig.update_traces(line_color='red')
        fig.update_layout(title = "Wind Speed by Date", title_x = 0.5)
    return fig

app.run(debug = True)